### Joining Tables
When working with Big Data, the data can be distributed among many sources or different tables. 

Therefore, when we are working with a given dataset A, but we need some data from dataset B to come up with an insight, joining tables comes in handy.

Let's work with some examples.

In [0]:
from pyspark.sql.functions import col, sum, min, max, countDistinct, mean
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

# Creating a dataframe for Sales Data
my_data1 = [("2023-01-01","PPDQ1","A",3626.3),
        ("2023-01-02","PPDQ2","A",389.5),
        ("2023-01-02","PPDQ1","B",2829.78),
        ("2023-01-03","PPDQ6","E",367.45),
        ("2023-01-04","PPDQ3","D",7266.45),
        ("2023-01-04","PPDQ3","D",703.7),
        ("2023-01-05","PPDQ3","F",4226.74),
        ("2023-01-05","PPDQ3","F",4226.74),
        ("2023-01-05","PPDQ3","E",426.74)
  ]

schema = StructType([ \
    StructField("date",StringType(),True), \
    StructField("product",StringType(),True), \
    StructField("store_id",StringType(),True), \
    StructField("total", DoubleType(), True)  ])
 
sales = spark.createDataFrame(data=my_data1,schema=schema)

# Creating a dataframe for Stores Data
my_data2 = [("A","NY","USA",2020),
        ("B","Rome","Italy",2023),
        ("C","Boston","USA",2021),
        ("D","NY","USA",2022),        
        ("F","Boston","USA",2022),
  ]

schema = StructType([ \
    StructField("store_id",StringType(),True), \
    StructField("city",StringType(),True), \
    StructField("country",StringType(),True), \
    StructField("opened_dt", IntegerType(), True)  ])

stores = spark.createDataFrame(data=my_data2,schema=schema)

In [0]:
display(sales)
display(stores)

date,product,store_id,total
2023-01-01,PPDQ1,A,3626.3
2023-01-02,PPDQ2,A,389.5
2023-01-02,PPDQ1,B,2829.78
2023-01-03,PPDQ6,E,367.45
2023-01-04,PPDQ3,D,7266.45
2023-01-04,PPDQ3,D,703.7
2023-01-05,PPDQ3,F,4226.74
2023-01-05,PPDQ3,F,4226.74
2023-01-05,PPDQ3,E,426.74


store_id,city,country,opened_dt
A,NY,USA,2020
B,Rome,Italy,2023
C,Boston,USA,2021
D,NY,USA,2022
F,Boston,USA,2022


In [0]:
display(
    sales
    .join(stores, on='store_id', how= 'left')
)

store_id,date,product,total,city,country,opened_dt
A,2023-01-01,PPDQ1,3626.3,NY,USA,2020
A,2023-01-02,PPDQ2,389.5,NY,USA,2020
B,2023-01-02,PPDQ1,2829.78,Rome,Italy,2023
E,2023-01-03,PPDQ6,367.45,null,null,null
D,2023-01-04,PPDQ3,7266.45,NY,USA,2022
D,2023-01-04,PPDQ3,703.7,NY,USA,2022
F,2023-01-05,PPDQ3,4226.74,Boston,USA,2022
F,2023-01-05,PPDQ3,4226.74,Boston,USA,2022
E,2023-01-05,PPDQ3,426.74,null,null,null


In [0]:
display(
    sales
    .join(stores, on='store_id', how= 'left_anti')
)

store_id,date,product,total
E,2023-01-03,PPDQ6,367.45
E,2023-01-05,PPDQ3,426.74


In [0]:
display(
    sales
    .join(stores, on='store_id', how= 'right')
)

store_id,date,product,total,city,country,opened_dt
A,2023-01-02,PPDQ2,389.5,NY,USA,2020
A,2023-01-01,PPDQ1,3626.3,NY,USA,2020
B,2023-01-02,PPDQ1,2829.78,Rome,Italy,2023
C,null,null,null,Boston,USA,2021
D,2023-01-04,PPDQ3,703.7,NY,USA,2022
D,2023-01-04,PPDQ3,7266.45,NY,USA,2022
F,2023-01-05,PPDQ3,4226.74,Boston,USA,2022
F,2023-01-05,PPDQ3,4226.74,Boston,USA,2022


In [0]:
display(
    sales
    .join(stores, on='store_id', how= 'inner')
)

store_id,date,product,total,city,country,opened_dt
A,2023-01-01,PPDQ1,3626.3,NY,USA,2020
A,2023-01-02,PPDQ2,389.5,NY,USA,2020
B,2023-01-02,PPDQ1,2829.78,Rome,Italy,2023
D,2023-01-04,PPDQ3,7266.45,NY,USA,2022
D,2023-01-04,PPDQ3,703.7,NY,USA,2022
F,2023-01-05,PPDQ3,4226.74,Boston,USA,2022
F,2023-01-05,PPDQ3,4226.74,Boston,USA,2022


In [0]:
display(
    sales
    .join(stores, on='store_id', how= 'full_outer')
)

store_id,date,product,total,city,country,opened_dt
A,2023-01-01,PPDQ1,3626.3,NY,USA,2020
A,2023-01-02,PPDQ2,389.5,NY,USA,2020
B,2023-01-02,PPDQ1,2829.78,Rome,Italy,2023
C,null,null,null,Boston,USA,2021
D,2023-01-04,PPDQ3,7266.45,NY,USA,2022
D,2023-01-04,PPDQ3,703.7,NY,USA,2022
E,2023-01-03,PPDQ6,367.45,null,null,null
E,2023-01-05,PPDQ3,426.74,null,null,null
F,2023-01-05,PPDQ3,4226.74,Boston,USA,2022
F,2023-01-05,PPDQ3,4226.74,Boston,USA,2022


In [0]:
display(
    sales
    .join(stores, sales.store_id == stores.store_id, 'inner')
)

date,product,store_id,total,store_id,city,country,opened_dt
2023-01-01,PPDQ1,A,3626.3,A,NY,USA,2020
2023-01-02,PPDQ2,A,389.5,A,NY,USA,2020
2023-01-02,PPDQ1,B,2829.78,B,Rome,Italy,2023
2023-01-04,PPDQ3,D,7266.45,D,NY,USA,2022
2023-01-04,PPDQ3,D,703.7,D,NY,USA,2022
2023-01-05,PPDQ3,F,4226.74,F,Boston,USA,2022
2023-01-05,PPDQ3,F,4226.74,F,Boston,USA,2022


In [0]:
display(
    sales
    .join(stores.select('store_id', 'city'), on='store_id', how='left')
    .groupBy('city')
    .agg(sum('total').alias('total_sales'))
    .withColumn('pct', F.round( col('total_sales')/sum('total_sales').over(Window.partitionBy()), 2))
)

city,total_sales,pct
NY,11985.95,0.5
Rome,2829.78,0.12
null,794.19,0.03
Boston,8453.48,0.35


Databricks visualization. Run in Databricks to view.